# Engineer
Here, we will engineer our features as follows:
* Convert numeric data to categorical
    * MoSold
    * MSSubClass
* Convert categorical data with orderings (likert scale type data) into ordinal data
E.g.,  `GarageQual`: NoGarage->0, Po->1, Fa->2, TA->3, Gd->4, Ex->5
* Create new features
    * Derived
    E.g., `NumFloors` in a property
    * Indicators (booleans)
    E.g., `IsPUD` whether a property is in a PUD
* Collapse number of categories for categorical features on a case-by-case basis
E.g., `OverallQual` has scale 1-10. 1-3 are simplified to "bad", 4-6 are "normal", 7-10 are "good"
* Dummify categorical features

In [327]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [328]:
import pandas as pd
import numpy as np
from helpers import data_dict
from helpers import features

In [329]:
df = pd.read_csv("../data/cleaned.csv")
df.drop('Unnamed: 0', inplace=True, axis=1)

## Convert numerical data to categorical
`YrSold`, `MoSold`, and `MSSubClass` are categorical data but are encoded as integers in the dataset

In [330]:
# NOTE: We will treat MoSold and YrSold as categorical because there are only 12 (Jan->Dec) and 5 values (2006->2010).
# This might not be the case if we had many years of sales data in our dataset
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df.MoSold.map(data_dict.convert_mosold)
df['MSSubClass'] = df['MSSubClass'].astype(str)

# Because we write out to a csv, we need to make sure that YrSold and MSSubClass are read back as categorical in other files.
# So we prepend a string to these columns
df['YrSold'] = df['YrSold'].apply(lambda x: "Yr_" + x)
df['MSSubClass'] = df['MSSubClass'].apply(lambda x: "Dwelling_" + x)

## Convert categorical to ordinal

In [331]:
# Overall features
# NO OP: These are already ordinalized

In [332]:
# Exterior features
df['ExterQual'] = df.ExterQual.map(data_dict.convert_exterqual)
df['ExterCond'] = df.ExterCond.map(data_dict.convert_extercond)

In [333]:
# Basement features
df['BsmtQual'] = df.BsmtQual.map(data_dict.convert_bsmtqual)
df['BsmtCond'] = df.BsmtCond.map(data_dict.convert_bsmtcond)
df['BsmtExposure'] = df.BsmtExposure.map(data_dict.convert_bsmtexposure)
df['BsmtFinType1'] = df.BsmtFinType1.map(data_dict.convert_bsmtfintype)
df['BsmtFinType2'] = df.BsmtFinType2.map(data_dict.convert_bsmtfintype)

In [334]:
# Home Interior features
df['Functional'] = df.Functional.map(data_dict.convert_functional)
df['FireplaceQu'] = df.FireplaceQu.map(data_dict.convert_fireplacequ)
df['HeatingQC'] = df.HeatingQC.map(data_dict.convert_heatingqc)
df['KitchenQual'] = df.KitchenQual.map(data_dict.convert_kitchenqual)

In [335]:
# Land features
df['LandSlope'] = df.LandSlope.map(data_dict.convert_landslope)
df['LotShape'] = df.LotShape.map(data_dict.convert_lotshape)

In [336]:
# Garage features
df['GarageCond'] = df.GarageCond.map(data_dict.convert_garagecond)
df['GarageQual'] = df.GarageQual.map(data_dict.convert_garagequal)

In [337]:
# Road features
df['Street'] = df.Street.map(data_dict.convert_street)
df['PavedDrive'] = df.PavedDrive.map(data_dict.convert_paveddrive)
df['Alley'] = df.Alley.map(data_dict.convert_alley)

In [338]:
# Other features
df['Utilities'] = df.Utilities.map(data_dict.convert_utilities)
df['PoolQC'] = df.PoolQC.map(data_dict.convert_poolqc)

## New Features
New data we can create from existing data. Does not mean we need to drop old data columns

In [339]:
# See whether a property is in a PUD from the dwelling type
df['IsPUD'] = df.MSSubClass.map(data_dict.get_pud_indicator)

In [340]:
# TODO LotShape isRegular

In [341]:
# Whether the property is near bad conditions (a busy road or a railroad)
df['IsNearNegativeCondition1'] = df.Condition1.map(data_dict.get_bad_condition_indicator)
df['IsNearNegativeCondition2'] = df.Condition2.map(data_dict.get_bad_condition_indicator)
df['IsNearNegativeCondition'] = df.apply(lambda x: x['IsNearNegativeCondition1'] | x['IsNearNegativeCondition2'], axis=1)
df.drop(['IsNearNegativeCondition1', 'IsNearNegativeCondition2'], axis=1, inplace=True)

# Whether the property is near good conditions (a park, green-belt, etc)
df['IsNearPositiveCondition1'] = df.Condition1.map(data_dict.get_good_condition_indicator)
df['IsNearPositiveCondition2'] = df.Condition2.map(data_dict.get_good_condition_indicator)
df['IsNearPositiveCondition'] = df.apply(lambda x: x['IsNearPositiveCondition1'] | x['IsNearPositiveCondition2'], axis=1)
df.drop(['IsNearPositiveCondition1', 'IsNearPositiveCondition2'], axis=1, inplace=True)

In [342]:
# Check good/bad counts
print(f"{df.IsNearNegativeCondition.sum()} properties near an artery or railroad")
print(f"{df.IsNearPositiveCondition.sum()} properties near park, green-belt, etc")

164 properties near an artery or railroad
56 properties near park, green-belt, etc


In [343]:
# Get number of floors for the property from the dwelling type
df['NumFloors'] = df.MSSubClass.map(data_dict.get_num_floors)

## Collapse/Combine Features
* Collapse: Use smaller scales for Likert scale-type categorical features (and drop the larger scale feature)
* Combine: Convert multiple features into a new feature (and drop the others)

Use smaller scales. Featured prefixed with `Collapse_`

In [344]:
df['Collapse_MSSubClass'] = df.MSSubClass.map(data_dict.collapse_mssubclass)

Combine multiple features. Features prefixed with `Combined_`

In [345]:
# Bathrooms
df['Combine_BathroomsBsmt'] = df['BsmtFullBath'] + df['BsmtHalfBath']*.5
df['Combine_BathroomsAbvGrd'] = df['FullBath'] + df['HalfBath']*0.5
df['AllBathrooms'] = df['Combine_BathroomsBsmt'] + df['Combine_BathroomsAbvGrd']

In [346]:
# Age (since remodeling, if property was remodeled)
df['tempYrSold'] = df['YrSold'].apply(lambda y: (y[3:])).astype(int)
df['tempWasRenovatedAfterBuilding'] = df.apply(lambda x: x['YearRemodAdd'] > x['YearBuilt'], axis=1)
df['Combine_Age'] = df.apply(lambda x: (x['tempYrSold'] - x['YearRemodAdd'])
                                        if x['tempWasRenovatedAfterBuilding']
                                        else (x['tempYrSold'] - x['YearBuilt']), axis=1)
# IsRenovated
df['IsRenovated'] = df.apply(lambda x: (x['YearRemodAdd'] > x['YearBuilt']), axis=1)
# Check
df[['Combine_Age', 'tempYrSold', 'YearBuilt', 'YearRemodAdd', 'IsRenovated']]

,Combine_Age,tempYrSold,YearBuilt,YearRemodAdd,IsRenovated
0,60,2010,1939,1950,True
1,25,2009,1984,1984,False
2,0,2007,1930,2007,True
3,6,2009,1900,2003,True
4,8,2009,2001,2001,False
...,...,...,...,...,...
2572,59,2009,1916,1950,True
2573,54,2009,1955,1955,False
2574,57,2007,1949,1950,True
2575,7,2007,2000,2000,False


In [347]:
df.drop(['tempYrSold', 'tempWasRenovatedAfterBuilding'], inplace=True, axis=1)

## Drop Other Features
Drop features that have been identified via EDA to be
* causes of multicolinearity
* have low explanatory power (insignificant; having low p-values)

In [348]:
# TODO drop features

## Add Log Target (SalePrice)
From EDA, we found that the distributions of features are skewed when plotted against `SalePrice`

In [349]:
df['LogSalePrice'] = np.log(df.SalePrice)

#### Housekeeping
Check that we organized every feature in `features.py`

In [350]:
# Test that we have all features labeled by type of data
features.check_features(df)

92 #columns == 92 #features+PID+targets


In [351]:
# Manually check that these are the same
features.check_features2(df)

,df.columns,features
0,1stFlrSF,1stFlrSF
1,2ndFlrSF,2ndFlrSF
2,3SsnPorch,3SsnPorch
3,AllBathrooms,AllBathrooms
4,Alley,Alley
...,...,...
87,Utilities,Utilities
88,WoodDeckSF,WoodDeckSF
89,YearBuilt,YearBuilt
90,YearRemodAdd,YearRemodAdd


Save outputs

In [352]:
df.to_csv("../data/engineered.csv")